In [ ]:
from joblib import dump
from os import listdir
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

import pandas as pd

In [ ]:
def add_get_int64_id(amenity):
	id = amenity.lower().replace(" ", "_").replace("/", "_")
	int64Ids.append(id)
	return "id=\"" + id + "\" name=\"" + id +"\""
def add_get_float64_id(amenity):
	id = amenity.lower().replace(" ", "_").replace("/", "_")
	float64Ids.append(id)
	return "id=\"" + id + "\" name=\"" + id +"\""
def removeLayer1(location):
	if not " > " in location:
		return " ".join(location.split()[1:])
	return " > ".join(location.split(" > ")[1:])

In [ ]:
for file in listdir("data"):
	int64Ids = []
	float64Ids = []
	data = pd.read_csv("data/" + file)
	for column in ["Identity", "Link"]:
		try:
			data.drop(column, axis = 1, inplace = True)
		except KeyError:
			pass
	for column in data.select_dtypes(include = "float64").columns:
		data = pd.concat([data, data.pop(column)], axis = 1)
	typeAndLocationOptions = [
		"<option value=\"" +
		x +
		"\">" +
		removeLayer1(x) +
		"</option>"
		for x in reversed(sorted(data["Location"].unique()))
	]
	data = pd.concat([data, pd.get_dummies(data.pop("Location").str.split(" > ", expand = True))], axis = 1)
	int64Options = [
		"<div class=\"col-6\"><label><b>" +
		x +
		":</b></label><select class=\"form-select\" " +
		add_get_int64_id(x) +
		" required><option value=\"No\">No</option><option value=\"Yes\">Yes</option></select></div>"
		for x in data.select_dtypes(include = "int64").columns
	]
	float64Options = [
		"<div class=\"col-6\"><label><b>" +
		x +
		":</b></label><input type=\"number\" class=\"form-control\" value=\"0\" min=\"0\" " +
		add_get_float64_id(x) +
		" required></div>"
		for x in data.drop(["Area", "Price"], axis = 1).select_dtypes(include = "float64")
	]
	boolColumns = list(data.select_dtypes(include = "bool").columns)



	x = data.drop("Price", axis = 1)
	y = data["Price"]
	trainX, testX, trainY, testY = train_test_split(x, y, test_size = 0.2, random_state = 1)



	modelA = Pipeline([
		("scaler", MinMaxScaler()),
		("model", GradientBoostingRegressor())
	])
	modelB = Pipeline([
		("scaler", MinMaxScaler()),
		("model", RandomForestRegressor())
	])



	modelA.fit(trainX, trainY)
	modelB.fit(trainX, trainY)



	scoreA = modelA.score(testX, testY) * 100
	scoreB = modelB.score(testX, testY) * 100



	print(file)
	print("Gradient Boosting Regressor:", "{:.2f}".format(scoreA) + "% Accuracy")
	print("Random Forest Regressor:", "{:.2f}".format(scoreB) + "% Accuracy")
	print()



	if scoreA > scoreB:
		modelF = Pipeline([
			("scaler", MinMaxScaler()),
			("model", GradientBoostingRegressor())
		])
	else:
		modelF = Pipeline([
			("scaler", MinMaxScaler()),
			("model", RandomForestRegressor())
		])



	modelF.fit(x, y)
	dump([modelF, data, int64Ids, float64Ids, typeAndLocationOptions, int64Options, float64Options, boolColumns], "../Module3/static/" + file[:-3] + "joblib")
	#data.info(verbose = True, memory_usage = True, show_counts = True)